In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import MiniBatchKMeans
from joblib import Parallel, delayed
import numpy as np
from tqdm import tqdm


def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out, indices_out = [], [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]
        indices_c = np.where(y == label)[0]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        n_sample = max(1, n_sample)

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])
        indices_out.append(indices_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0), np.concatenate(indices_out, axis=0)


def compute_uncertainties(model, X_pool, batch_size=10000):
    def predict_batch(X_chunk):
        probs = model.predict_proba(X_chunk)
        return 1 - np.max(probs, axis=1)

    chunks = [X_pool[i:i + batch_size] for i in range(0, len(X_pool), batch_size)]
    uncertainty_chunks = Parallel(n_jobs=-1)(delayed(predict_batch)(chunk) for chunk in tqdm(chunks, desc="Computing uncertainties"))
    return np.concatenate(uncertainty_chunks)


def batch_mode_active_learning(X_pool, y_pool, model, fraction=0.2, random_state=42, max_clusters_per_iter=1000, max_top_k=10000, batch_size=10000):
    rng = np.random.RandomState(seed=random_state)
    n_total = len(X_pool)
    n_select = max(1, int(n_total * fraction))
    selected_indices = []

    if hasattr(X_pool, 'toarray'):
        X_pool = X_pool.toarray()

    classes = np.unique(y_pool)

    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        if len(idx_c) > 0:
            selected_idx = rng.choice(idx_c, size=1)[0]
            selected_indices.append(selected_idx)

    uncertainties = compute_uncertainties(model, X_pool, batch_size)

    n_remaining = n_select - len(selected_indices)
    current_indices = np.setdiff1d(np.arange(n_total), selected_indices)

    pbar = tqdm(total=n_select, desc="Selecting samples (active learning)")
    pbar.update(len(selected_indices))

    while n_remaining > 0 and len(current_indices) > 0:
        n_clusters = min(max_clusters_per_iter, n_remaining, len(current_indices))
        if n_clusters == 0:
            break

        top_k = min(n_clusters * 5, len(current_indices) - 1, max_top_k)
        if top_k <= 0:
            break

        current_uncertainties = uncertainties[current_indices]
        top_indices = np.argpartition(-current_uncertainties, top_k)[:top_k]
        top_global_indices = current_indices[top_indices]
        X_top = X_pool[top_global_indices]

        n_clusters = min(n_clusters, len(X_top))
        if n_clusters == 0:
            break

        kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=random_state, batch_size=1000, max_iter=20, init='k-means++')
        kmeans.fit(X_top)

        # Vectorized distance computation
        X_sq = np.sum(X_top ** 2, axis=1, keepdims=True)
        C_sq = np.sum(kmeans.cluster_centers_ ** 2, axis=1)
        XC = X_top @ kmeans.cluster_centers_.T
        dists = np.sqrt(np.maximum(X_sq - 2 * XC + C_sq, 0))
        cluster_closest = np.argmin(dists, axis=0)

        selected_indices.extend(top_global_indices[cluster_closest])

        pbar.update(len(cluster_closest))
        
        mask = ~np.isin(current_indices, top_global_indices[cluster_closest])
        current_indices = current_indices[mask]
        n_remaining -= n_clusters
    pbar.close()

    selected_indices = np.array(selected_indices[:n_select])
    X_selected = X_pool[selected_indices]
    y_selected = y_pool[selected_indices]
    remaining_idx = np.setdiff1d(np.arange(n_total), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    selected_class_counts = np.bincount(y_selected, minlength=len(classes))
    missing_classes = classes[selected_class_counts == 0]
    if len(missing_classes) > 0:
        print(f"Warning: Missing classes {missing_classes}. Adding one sample per missing class.")
        for c in missing_classes:
            idx_c = np.where(y_pool == c)[0]
            if len(idx_c) > 0:
                selected_idx = rng.choice(idx_c, size=1)[0]
                selected_indices = np.append(selected_indices, selected_idx)
        selected_indices = selected_indices[:n_select]
        X_selected = X_pool[selected_indices]
        y_selected = y_pool[selected_indices]
        remaining_idx = np.setdiff1d(np.arange(n_total), selected_indices)
        X_rest = X_pool[remaining_idx]
        y_rest = y_pool[remaining_idx]

    print(f"Selected samples (fraction={fraction}): {len(y_selected)}")
    print(f"Label distribution: {np.bincount(y_selected, minlength=len(classes))}")

    return X_rest, X_selected, y_rest, y_selected


# ==== Usage ====
# Replace with your actual data
X_pool = x_test_adv_0_1
y_pool = y_label_0_1

X_train_init, y_train_init, _ = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)

clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(X_train_init, y_train_init)

_, X_1, _, y_1 = batch_mode_active_learning(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.01, random_state=42
)


Initial labeled set size: 214873
Initial label distribution: [95648     1 48124  1059   484 46858     1 22374    65   257     2]


In [ ]:
X_1.shape


In [ ]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt1 = DecisionTreeClassifier()
# dt1.fit(X_1, y_1)

In [ ]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_BatchMode/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_BatchMode/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

In [ ]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

In [ ]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt1.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning_BatchMode/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning_BatchMode/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [ ]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_ActiveLearning_BatchMode/WUSTL_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [ ]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_ActiveLearning_BatchMode.csv")

In [18]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB1,0,4,2,0,0,225386,0,13360,1,0,0,238753
1,XGB1,439197,0,271005,0,21,1042,0,3128,1046,820,0,716259
2,XGB1,447881,0,251129,34,672,11206,0,3410,378,1549,0,716259
3,XGB1,439197,0,271005,0,21,1042,0,3128,1046,820,0,716259
4,XGB1,91,0,7446,32,7764,689317,5,1734,106,9764,0,716259
5,XGB1,170633,3,507053,71,150,28760,0,7540,1637,210,202,716259
6,XGB1,0,9,9,0,0,676163,0,40075,3,0,0,716259
7,XGB1,66522,0,7781,34856,1858,61162,0,541403,2552,110,15,716259
8,XGB1,3802,0,710693,0,0,584,0,501,679,0,0,716259
9,XGB1,437202,1,273276,5,23,3687,0,768,826,471,0,716259
